In [ ]:
import seaborn as sns 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import requests
import sqlite3

api_key = "RGAPI-ee3c1e70-277f-40a1-9917-7704ab5fb21d"
divisions =["I","II","III","IV"]
tiers = ["IRON","BRONZE","SILVER","GOLD","PLATINUM", "EMERALD", "DIAMOND","MASTER","GRANDMASTER","CHALLENGER"]

players = []

for tier in tiers[0:7]:
    for division in divisions: 
            #print(f"Getting data for rank and division: {i} {j}")
            url = f"https://na1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{tier}/{division}?page=1&api_key={api_key}"
            response = requests.get(url)
            response.raise_for_status()

            entries = response.json()

            for player in entries:
                players.append((player['puuid'], division, tier))

In [ ]:
def get_connection():
    return sqlite3.connect('league.db')

In [ ]:

def init_db():
    '''Initialize the SQLite database and create the 
        players table if it doesn't exist
    '''
    conn = get_connection()
    cur = conn.cursor()
    #checks for database connection. Returns error if not connected
    try: 
        conn.execute('SELECT 1;')
        cur.close()
        return True
    except Exception as e:
        cur.close()
        return False
init_db()

True

In [ ]:
def collect_puuids():
    with get_connection() as conn:
        cur = conn.cursor()
        cur.execute('''
                CREATE TABLE IF NOT EXISTS players (
                    puuid TEXT PRIMARY KEY,
                    division TEXT,
                    tier TEXT)''')

        for puuid, tier, division in players:
            cur.execute('''
                INSERT OR IGNORE INTO players (puuid, division, tier)
                        VALUES (?, ?, ?)
                ''', (puuid, division, tier))
        conn.commit()
        cur.execute("SELECT * FROM players LIMIT 20;")
        rows = cur.fetchall()
        for row in rows:
            print(row)
collect_puuids()

('w0cZ6V2GCZY4_J7Z9AEn6xjTtTKdc00gU9pKVoicKwEXYFQNdRgOYjcwsOmg-UPrtS-X4rsPLLYe9Q', 'IRON', 'I')
('AafwEK7Ufn4AQed3lTcMBWlcip9v0_jvJALRkEmAblSYqYg-AWbW0o7CSGlVRAO2bRXUpRNt9tOL8g', 'IRON', 'I')
('KWs8SDO31IoesoiXEptB3UYfal88zFLZAWio5KLMzuHZ-X5abUSjkj8rXhu7yK5j_Bci8FBy_oiADA', 'IRON', 'I')
('kJp5d3JUiQAgDlhAWdSMwdtX1LcFb-YfcT_RmgVurrbE7ItsivbzHitXs2Mx9V44eo9QCdZDVdQYPg', 'IRON', 'I')
('s5ycRqpmU_06S7ODroKm2GukywTrLCpJ4lymwmlXp7wnN94_K8DZq9cFPj0L71ttI7Bqb2L6YL90PQ', 'IRON', 'I')
('Og4z3W8C25aO2IpWM9BDULySg1KgJHDT5whLkHJiHOESqU4GgjJCbgb2z5I7zJLHivtH3fz_Y3dxLA', 'IRON', 'I')
('un92yQgF5Ackckhmkl6zya6R8G0efSb-jWBJjtyzdyy1dSAKGzn7Rb8cNcImupamOhzMVHwGNy01CQ', 'IRON', 'I')
('KHVMoMd0pL8_8GU1JkNKxG0pHQjpWKwh8N-lT8jQTIgkeol05cLOV7C1yX08PsuKCwpZ6PLEwabijg', 'IRON', 'I')
('psNqL9VK5uzktQgMVhMOD7SVDKh8ABzsUf9ePko7EH-BKVtpteePRCt24dN-L0wFved9zUjixWt2iA', 'IRON', 'I')
('o2h_jv5S2kIPwly6WGxG_D5N6CEQkGUUTd15J9_3q3bV1XL4Xcqy3Qn_IcL4OBtrD-bCpAqOUkWXkA', 'IRON', 'I')
('QtoVBLoUeqTXW8IQFPrgjCrWg0y6kiRMLDOcmr

In [ ]:
def collect_matches():
    with get_connection() as conn:
        matches = []
        cur = conn.cursor()
        cur.execute('SELECT puuid FROM players')
        puuids = cur.fetchall()
        for (puuid,) in puuids:
            url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue=420&start=0&count=20&api_key={api_key}"
            reponse = requests.get(url)
            reponse.raise_for_status()
            match_ids = reponse.json()
            for match_id in match_ids:
                matches.append((puuid, match_id))
collect_matches()

            